# ![Specutils: Un paquete de Astropy para espectroscopía](data/specutils_logo.png)

Este notebook proporciona una visión general del paquete coordinado de Astropy llamado **Specutils**. Aunque este notebook está pensado como una introducción interactiva a *specutils* en el momento de su creación, la fuente oficial de información sobre el paquete es la documentación de la versión más reciente:

https://specutils.readthedocs.io

# Instalación

<div class="alert alert-block alert-warning">
Si seguiste las instrucciones del taller, es probable que **specutils** ya esté instalado. Pero dependiendo de cuándo lo instalaste, puede que necesites seguir las <a href="../../edit/00-Install_and_Setup/UPDATING.md">instrucciones en el archivo UPDATING.md</a> ubicado en la raíz de este repositorio.
</div>

Si aún no está instalado, instalar `specutils` debería ser tan sencillo como ejecutar uno de los siguientes comandos:

```
$ conda install -c conda-forge specutils
```

o también,

```
$ pip install specutils
```

Aunque para más información, consulta la [sección de instalación en la documentación](https://specutils.readthedocs.io/en/latest/installation.html).

Una vez que estos paquetes están instalados, ya es posible realizar las importaciones fundamentales necesarias para este notebook:

In [ ]:
import numpy as np

import astropy.units as u
from astropy.coordinates import SpectralCoord

import specutils
from specutils import Spectrum1D
specutils.__version__

In [ ]:
# para graficar:
%matplotlib inline
import matplotlib.pyplot as plt

# para mostrar automáticamente las unidades de cantidad en los ejes:
from astropy.visualization import quantity_support
quantity_support();

# Fundamentos de specutils

## Objetos para representar espectros

El propósito más fundamental de `specutils` es contener las estructuras de datos compartidas a nivel Python para almacenar espectros astronómicos. Es importante reconocer que esto no es lo mismo que la representación *en disco*. Como se describe más adelante, specutils provee cargadores y escritores para varias representaciones en disco, con la intención de que todas carguen a un conjunto común de interfaces en memoria/Python. 

Esas interfaces (implementadas como clases de Python) se describen en detalle en la [sección relevante de la documentación](https://specutils.readthedocs.io/en/latest/types_of_spectra.html), que contiene este diagrama:

![Clases de Specutils](data/specutils_classes_diagrams.png)

El principio central es que todas estas representaciones contienen un atributo `spectral_axis` así como un atributo `flux` (y opcionalmente una `uncertainty` correspondiente). El primero suele ser longitud de onda para espectros ópticos, infrarrojos y radio (OIR), pero podría ser frecuencia o energía para espectros de radio o rayos X. Independientemente del tipo de eje espectral usado, la clase intenta interpretarlo apropiadamente, usando las características de `astropy.Quantity` para distinguir entre tipos de ejes.

De manera similar, `flux` puede o no tener una unidad tradicional astronómica (ej. Jy o erg s$^{-1}$ cm$^{-2}$ Å$^{-1}$), pero se trata como la porción del espectro que actúa como flujo. 

Las distintas clases se diferencian según si estos atributos son unidimensionales o no, y cómo se mapea la dimensionalidad del `spectral_axis` sobre el `flux`. 

El caso más simple (y el que se considera principalmente aquí) es el escalar `Spectrum1D`, que es un único espectro con un `flux` y un `spectral_axis` de tamaño coincidente.

## Conceptos básicos para crear objetos Spectrum1D

Aunque frecuentemente un espectro se cargará desde un archivo (ver más abajo), si el formato no es compatible o se requiere una personalización particular, los espectros pueden crearse directamente a partir de arreglos y objetos `Quantity` de Astropy. Por ejemplo:

In [ ]:
# Crear un flujo simulado con ruido alrededor de 2 erg/s/cm²/Å
flux = (np.random.randn(100)*0.2 + 2) * u.erg / u.s / (u.cm**2) / u.AA 

# Crear un arreglo de longitudes de onda linealmente espaciadas de 3000 a 8000 Å
wavelength = np.linspace(3000, 8000, 100)*u.AA

# Crear un objeto Spectrum1D con el eje espectral y el flujo definidos arriba
spec1d = Spectrum1D(spectral_axis=wavelength, flux=flux)

# Graficar el espectro usando un gráfico de escalones
plt.step(spec1d.spectral_axis, spec1d.flux)

# Mostrar el objeto Spectrum1D
spec1d

Estos pueden transformarse fácilmente a otras unidades espectrales si se desea:

In [ ]:
# Convertir el espectro a unidades de Jansky (Jy)
jyspec1d = spec1d.with_flux_unit(u.Jy)

# Graficar el flujo contra la frecuencia en las nuevas unidades
plt.plot(jyspec1d.frequency, jyspec1d.flux)

## Trabajando con ejes espectrales

Creamos `Spectrum1D` simplemente como arreglos de tipo Quantity, por lo que pueden ser tratados igual que objetos `Quantity` cuando sea conveniente, como para conversiones de unidades y cosas similares:

In [ ]:
spec1d.spectral_axis

In [ ]:
spec1d.spectral_axis.to(u.nm)

In [ ]:
spec1d.spectral_axis.to(u.THz, u.spectral())

Pero internamente estos son WCS (Sistemas de Coordenadas del Mundo) completamente funcionales que siguen la interfaz WCS de [Astropy APE14](https://github.com/astropy/astropy-APEs/blob/main/APE14.rst) junto con el paquete [GWCS](https://gwcs.readthedocs.io/). Por lo tanto, puedes usar esto para hacer conversiones entre ejes espectrales y de píxeles:

In [ ]:
spec1d.wcs.pixel_to_world([10, 10.5])

In [ ]:
spec1d.wcs.world_to_pixel(SpectralCoord([400, 450]*u.nm))

## Incertidumbres

Actualmente, la forma más compatible de usar incertidumbres es mediante la infraestructura diseñada para el objeto `astropy.nddata` (aunque en muchos casos simplemente pasar un arreglo de incertidumbre también funcionará):

In [ ]:
from astropy.nddata import StdDevUncertainty

# Creamos una incertidumbre estándar constante para cada punto del espectro
unc = StdDevUncertainty(0.2 * np.ones(100))

# Creamos un objeto Spectrum1D con la incertidumbre incluida
spec1d_unc = Spectrum1D(spectral_axis=wavelength, flux=flux, uncertainty=unc)

# Graficamos el espectro con barras de error usando matplotlib,
# extrayendo los valores numéricos de las cantidades para compatibilidad
plt.errorbar(spec1d_unc.wavelength.value, 
             spec1d_unc.flux.value, 
             spec1d_unc.uncertainty.array,
             fmt='-',
             drawstyle='steps-mid',
             ecolor='k')

### Ejercicio

Crea un objeto `Spectrum1D` para un cuerpo negro ideal a 5800 K y grafícalo. Luego intenta lo mismo, pero añadiendo ruido (aleatorio) y almacenándolo como incertidumbre.

Pista: aunque puedes hacerlo manualmente si conoces la función de Planck, hay una función en Astropy que te puede ayudar con esto — puedes encontrarla buscando en la [documentación de Astropy](http://docs.astropy.org).

## Cargando espectros desde archivos

Specutils también incluye lectores para una variedad de formatos de datos espectrales (incluyendo cargadores para futuros instrumentos del JWST). Aunque el soporte para formatos específicos depende principalmente de usuarios (¡como tú!) que provean lectores, puede que ya exista uno implementado para tu formato de espectro favorito. Como ejemplo, consideramos un espectro del [Sloan Digital Sky Survey](http://skyserver.sdss.org/):

In [ ]:
from urllib.request import urlretrieve

# Descargar el archivo FITS del espectro SDSS desde la URL proporcionada y guardarlo localmente en 'data/sdss_spectrum.fits'
url = 'https://data.sdss.org/sas/dr16/sdss/spectro/redux/26/spectra/1323/spec-1323-52797-0012.fits'
urlretrieve(url, 'data/sdss_spectrum.fits')

In [ ]:
# Leer el espectro SDSS desde el archivo FITS usando el lector compatible con SDSS-III/IV
sdss_spec = Spectrum1D.read('data/sdss_spectrum.fits', format='SDSS-III/IV spec')

# Graficar el espectro: eje x con la longitud de onda y eje y con el flujo
plt.plot(sdss_spec.wavelength, sdss_spec.flux)

Para ver la lista completa de formatos que puedes leer con tu versión actual de specutils, consulta la tabla al final del método `Spectrum1D.read`:

In [ ]:
help(Spectrum1D.read)

### Ejercicio

Si tienes tus propios datos espectroscópicos, intenta cargar un archivo aquí usando alguno de los cargadores incorporados o la interfaz `Spectrum1D`, y gráficalo. Si no tienes datos propios a la mano, intenta descargar algo de interés desde un archivo público (por ejemplo, datos públicos del HST usando MAST) y cárgalo.

# Manipulación de espectros

Además de las herramientas de análisis descritas con más detalle en [el siguiente cuaderno](Specutils_analisis.ipynb), Specutils también proporciona funcionalidades para manipular espectros. En general, estas siguen el patrón de crear *nuevos* objetos de specutils con los resultados de la operación en lugar de modificar los objetos en el lugar.

La operación más sencilla son las manipulaciones aritméticas. En general, estas siguen los patrones esperados. Por ejemplo:

In [ ]:
# Restamos un valor constante de flujo a todo el espectro original
newspec1d = spec1d - 2 * u.erg / u.s / (u.cm**2) / u.AA

# Graficamos el nuevo espectro resultante
plt.step(newspec1d.wavelength, newspec1d.flux)

Sin embargo, cuando hay ambigüedad en tu intención — por ejemplo, con dos espectros que tienen diferentes unidades y no está claro cuál es el resultado esperado — generalmente se generan errores en lugar de que el código intente adivinar.

In [ ]:
# Esto genera un eroor:
newspec1d - jyspec1d 

Resolver esto requiere una conversión explícita:

In [ ]:
spec1d.with_flux_unit(u.Jy) - jyspec1d

### Ejercicio

Toma el espectro de cuerpo negro que generaste arriba y crea una "característica espectral" agregando una línea de absorción o emisión gaussiana usando los operadores aritméticos demostrados anteriormente.  
(Pista: [astropy.modeling](http://docs.astropy.org/en/stable/modeling/) contiene [una implementación del perfil de línea Gaussiana](http://docs.astropy.org/en/stable/api/astropy.modeling.functional_models.Gaussian1D.html#astropy.modeling.functional_models.Gaussian1D) que te puede ser útil.)

## Suavizado (Smoothing), etc.

Existen herramientas de manipulación más complejas, descritas en la [sección relevante de la documentación](https://specutils.readthedocs.io/en/latest/manipulation.html). Como ejemplo final de esto, suavizamos nuestro espectro de ejemplo usando suavizado gaussiano:

In [ ]:
from specutils.manipulation import gaussian_smooth

# Suavizado gaussiano con un núcleo gaussiano de 0.75 píxeles
smoothed_spec = gaussian_smooth(spec1d, 0.75)
plt.step(smoothed_spec.wavelength, smoothed_spec.flux)

### Ejercicio

Intenta suavizar tu espectro cargado en el ejemplo anterior (o el espectro SDSS). Compara todos los tipos de núcleos disponibles y decide cuál parece ser el más apropiado para tu espectro.

# Próximos pasos

Mientras que lo anterior se centra en la carga y visualización de espectros, el siguiente cuaderno está enfocado en el análisis científico cuantitativo de estos espectros. Ve al [cuaderno de análisis de Specutils](Specutils_analysis.ipynb) para más información al respecto.